In [18]:
# Dataset : https://www.kaggle.com/datasets/gauravduttakiit/ants-bees?resource=download
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader

In [19]:
transforms={'train':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),
    'val':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [20]:
data_dir='data/archive/hymenoptera_data'
#image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),transform=transforms[x]) for x in ['train','val']}


In [21]:
image_datasets={}
image_datasets['train']=datasets.ImageFolder(os.path.join(data_dir,'train'),transform=transforms['train'])
image_datasets['val']=datasets.ImageFolder(os.path.join(data_dir,'val'),transform=transforms['val'])

In [22]:
data_loader={}
data_loader['train']=DataLoader(image_datasets['train'],batch_size=32,shuffle=True)
data_loader['val']=DataLoader(image_datasets['val'],batch_size=32,shuffle=True)

In [23]:
model=models.resnet18(pretrained=True)
#print(model)
model_ftrs=model.fc.in_features
model_ft=nn.Linear(model_ftrs,out_features=2)
criterian=nn.CrossEntropyLoss()
optimizer=optim.Adam(model_ft.parameters(),lr=0.0001)
model.fc = model_ft

f:\22EG107B03\DL-Lab\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
f:\22EG107B03\DL-Lab\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
for epoch in range(50):
    for input,label in data_loader['train']:
        output=model(input)
        loss=criterian(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(loss.item())


0.4120267331600189
0.25006264448165894
0.4869869649410248
0.26737672090530396
0.2910652756690979
0.32185181975364685
0.36096104979515076
0.4434998631477356
0.24451029300689697
0.3062868118286133
0.22286593914031982
0.3160650134086609
0.19962038099765778
0.23654906451702118
0.32680580019950867
0.2748923599720001
0.2854994535446167
0.20763568580150604
0.20000331103801727
0.22996871173381805
0.3560146689414978
0.21603135764598846
0.2773166000843048
0.19440782070159912
0.1924826055765152
0.2515133321285248
0.22710900008678436
0.14036571979522705
0.26123690605163574
0.29925909638404846
0.21766945719718933
0.21949073672294617
0.20915687084197998
0.4232669472694397
0.23916879296302795
0.22005900740623474
0.19258061051368713
0.24240367114543915
0.44993600249290466
0.29693013429641724
0.2579004466533661
0.15441212058067322
0.22266796231269836
0.2684329152107239
0.3039762079715729
0.2448691576719284
0.24125227332115173
0.32544785737991333
0.3192668557167053
0.14382532238960266


In [29]:
model.eval()
with torch.no_grad():
    correct=0
    total=0
    for input,label in data_loader['val']:
        output=model(input)
        _,predicted=torch.max(output.data,1)
        total+=label.size(0)
        correct+=(predicted==label).sum().item()
    print('Accuracy: {}%'.format(100*correct/total))
    print(total, correct)

Accuracy: 86.9281045751634%
153 133
